In [1]:
from sqlmodel import SQLModel, Field, Session, create_engine, select
from typing import List, Optional
from sqlmodel import Relationship

import pandas as pd

from build_db.models import *

# Create the database engine
engine = create_engine("sqlite:///tmqm.db")



In [ ]:
from sqlmodel import select

# Define the session to interact with the database
with Session(engine) as session:
    # Create a SQL query to select structures where 'stoichometry' contains "La" and there are more than 100 atoms
    query = select(Structure).where(Structure.stoichometry.contains("La")).where(Structure.num_atoms > 100) 

    # print the constructed sql query
    print(query)
    
    # Execute the query and fetch the results
    results = session.exec(query).all()

    # Make a dataframe with the results
    records = [i.model_dump() for i in results] # i.model_dump results in a dictionary
    df = pd.DataFrame.from_records(records)

    # Get all of the properties
    props = []
    for structure in results:
        #print(f"ID: {structure.id}, Stoichometry: {structure.stoichometry}, Num Atoms: {structure.num_atoms}")

        # we can retrieve properties this way because of relationships we set up on the models
        # this would be more complicated if there was more than one set of coordinates per structure
        props.extend( p.model_dump() for p in structure.coordinates[0].properties) 

df.info()

In [ ]:
import pandas as pd

df2 = pd.DataFrame(props)
df2.info()

In [ ]:
df.head()

In [ ]:
df2.head()

In [ ]:
df2.sort_values(by=['coordinates_id', 'property'], inplace=True)

# Pivot the DataFrame, keeping 'coordinates_id' as the row identifier
wide_df = df2.pivot_table(index='coordinates_id', columns='property', values='value', aggfunc='first').reset_index()

# Reset the column names
wide_df.columns.name = None  # Remove the 'property' label from the columns

# Display the result
wide_df.head()


In [ ]:
# Print an xyz file
print(results[0].coordinates[0].xyz)

In [ ]:
import py3Dmol

xyzview = py3Dmol.view(width=400,height=400)
xyzview.addModel(results[0].coordinates[0].xyz,'xyz')
xyzview.setStyle({'stick':{}})
xyzview.setBackgroundColor('0xeeeeee')


xyzview.zoomTo()
xyzview.show()